# Analyze visually evoked responses (ERPs) 

## Set-up

#### Set path

In [ ]:
# Define paths
REPO_PATH = r"C:\Users\micha\projects\v1_v4_1024_elec" # set this
PROJECT_PATH = r'G:\Shared drives\v1_v4_1024'


#### Imports

In [ ]:
# general
import numpy as np
import matplotlib.pyplot as plt
import os
import odml
from scipy.signal import find_peaks, peak_widths

# custom
import sys
sys.path.append(REPO_PATH)
from scripts.plots import plot_schematic

#### Settings

In [ ]:
# Session of interest (will ultimately apply to all sessions)
SESSIONS = ["A_SNR_140819", "L_SNR_250717"]
SESSION = "A_SNR_140819"

# Dataset details
FS = 500 # sampling frequency

#### Functions

## Main

#### Create output directory

In [ ]:
dir_out = f"{PROJECT_PATH}/figures/erp"
if not os.path.exists(dir_out):
    os.makedirs(dir_out)

#### load data

In [ ]:
# load example LFP epochs
dir_input = fr"G:\Shared drives\v1_v4_1024\data\lfp\lfp_epochs\{SESSION}"
fname_in = # load full epoch
lfp = np.load()
print(f"LFP Shape: \t{lfp.shape} (trials x channels x samples)")

In [ ]:
# load metadata for example sessions
odml_path = fr"{PROJECT_PATH}\data\dataset\{SESSION}\metadata_{SESSION}.odml"
metadata = odml.load(odml_path)

### Compute ERP

In [ ]:
# average over trials
erp = np.mean(lfp, axis=0)
print(f"ERP Shape: \t{erp.shape} (channels x samples)")

### Plot ERP

In [ ]:
# plot erp

# init time-vector
time = np.arange(-0.5, 0.5, 1/FS) * 1000


### Quantify ERP 

In [ ]:
# compute ERP parameters (latency, amplitude, width) for grand-average ERP (*first peak only)


In [ ]:
# define function to define params

# def comp_erp_params():



In [ ]:
# compute for each channel


In [ ]:
# plot schematic of ERP params


### Repeat across subjects